In [1]:
import os
import sys
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyts.image import GramianAngularField

In [2]:
if sys.platform == "win32":
    dataPath = r"C:\\Users\\KOTC\\Desktop\\0919\\Test2\\"
elif sys.platform == "darwin":
    dataPath = "/Users/jwh/Desktop/0913/Test2/result/"

fileNames = [file for file in sorted(glob.glob(dataPath + "*.csv")) if os.path.isfile(file)]

In [3]:
sensorNames = [f"s{i}" for i in np.arange(1, 7, 1)]

numRows = 4000

In [4]:
classData = fileNames[1:]
classData

[]

In [7]:
files = [pd.read_csv(file, sep= ",", nrows= numRows) for file in classData]

In [8]:
base = pd.read_csv(fileNames[0], sep= ",", nrows= numRows)
meanBase = pd.DataFrame(base.mean(axis= 0), columns= ['base'])

meanBase.head()

,base
s1,-828.26300
s2,-854.03200
s3,-886.97450
s4,756.29175
s5,744.35150


In [9]:
def calibration(df):
    res = [df.iloc[:, i] - meanBase.iloc[i, 0] for i in range(6)]
    res = np.array(res).reshape(6, -1).T
    res = pd.DataFrame(res, columns= sensorNames)
    return res

In [10]:
calibrated = [calibration(files[i]) for i in range(len(files))]
calibrated = np.array(calibrated)
calibrated = calibrated.reshape(27, 6, 4000, 1)
print(calibrated.shape)

(27, 6, 4000, 1)


In [12]:
splitedCase = []
for i in range(calibrated.shape[0]):
    splitedSensor = []
    for j in range(calibrated.shape[1]):
        splitedData = []
        for k in range(int(len(calibrated[i, j]) / 16)):
            splited = calibrated[i, j, k * 16 : (16 + (k * 16)), :]
            splitedData.append(splited)
        splitedData = np.array(splitedData)
        splitedSensor.append(splitedData)
    splitedSensor = np.array(splitedSensor)
    splitedCase.append(splitedSensor)
splitedCase = np.array(splitedCase)

print(splitedCase.shape)

(27, 6, 250, 16, 1)


In [13]:
gaf = GramianAngularField(method= 'difference')

In [14]:
encodedResult = []
for i in range(splitedCase.shape[0]):
    encodedSensor = []
    for j in range(splitedCase.shape[1]):
        encodedData = []
        for k in range(splitedCase.shape[2]):
            data = splitedCase[i, j, k, :]
            encoded = gaf.fit_transform(data.T)
            encoded = encoded.reshape(16, 16)
            encodedData.append(encoded)
        encodedData = np.array(encodedData)
        encodedSensor.append(encodedData)
    encodedSensor = np.array(encodedSensor)
    encodedResult.append(encodedSensor)
encodedResult = np.array(encodedResult)

print(encodedResult.shape)

(27, 6, 250, 16, 16)


In [15]:
concatenateResult = []
for c in range(encodedResult.shape[0]):
    sensor1 = encodedResult[c, 0]
    sensor2 = encodedResult[c, 1]
    sensor3 = encodedResult[c, 2]
    sensor4 = encodedResult[c, 3]
    sensor5 = encodedResult[c, 4]
    sensor6 = encodedResult[c, 5]

    concatenateImg = []
    for d in range(encodedResult.shape[2]):
        sensor1Img = sensor1[d].reshape(16, 16, 1)
        sensor2Img = sensor2[d].reshape(16, 16, 1)
        sensor3Img = sensor3[d].reshape(16, 16, 1)
        sensor4Img = sensor4[d].reshape(16, 16, 1)
        sensor5Img = sensor5[d].reshape(16, 16, 1)
        sensor6Img = sensor6[d].reshape(16, 16, 1)

        res = np.concatenate((sensor1Img, sensor2Img), axis= 2)
        res = np.concatenate((res, sensor3Img), axis= 2)
        res = np.concatenate((res, sensor4Img), axis= 2)
        res = np.concatenate((res, sensor5Img), axis= 2)
        res = np.concatenate((res, sensor6Img), axis= 2)

        concatenateImg.append(res)

    concatenateImg = np.array(concatenateImg)
    concatenateResult.append(concatenateImg)
concatenateResult = np.array(concatenateResult)

print(concatenateResult.shape)

(27, 250, 16, 16, 6)


In [16]:
c1Idx = [0, 1, 2]
c2Idx = [3, 4, 5]
c3Idx = [6, 7, 8]
c4Idx = [9, 10, 11]
c5Idx = [12, 13, 14]
c6Idx = [15, 16, 17]
c7Idx = [18, 19, 20]
c8Idx = [21, 22, 23]
c9Idx = [24, 25, 26]

concateClass1 = [concatenateResult[idx] for idx in c1Idx]
concateClass2 = [concatenateResult[idx] for idx in c2Idx]
concateClass3 = [concatenateResult[idx] for idx in c3Idx]
concateClass4 = [concatenateResult[idx] for idx in c4Idx]
concateClass5 = [concatenateResult[idx] for idx in c5Idx]
concateClass6 = [concatenateResult[idx] for idx in c6Idx]
concateClass7 = [concatenateResult[idx] for idx in c7Idx]
concateClass8 = [concatenateResult[idx] for idx in c8Idx]
concateClass9 = [concatenateResult[idx] for idx in c9Idx]

concateClass1 = np.array(concateClass1)
concateClass2 = np.array(concateClass2)
concateClass3 = np.array(concateClass3)
concateClass4 = np.array(concateClass4)
concateClass5 = np.array(concateClass5)
concateClass6 = np.array(concateClass6)
concateClass7 = np.array(concateClass7)
concateClass8 = np.array(concateClass8)
concateClass9 = np.array(concateClass9)

print("Class1: ", concateClass1.shape)
print("Class2: ", concateClass2.shape)
print("Class3: ", concateClass3.shape)
print("Class4: ", concateClass4.shape)
print("Class5: ", concateClass5.shape)
print("Class6: ", concateClass6.shape)
print("Class7: ", concateClass7.shape)
print("Class8: ", concateClass8.shape)
print("Class9: ", concateClass9.shape)


Class1:  (3, 250, 16, 16, 6)
Class2:  (3, 250, 16, 16, 6)
Class3:  (3, 250, 16, 16, 6)
Class4:  (3, 250, 16, 16, 6)
Class5:  (3, 250, 16, 16, 6)
Class6:  (3, 250, 16, 16, 6)
Class7:  (3, 250, 16, 16, 6)
Class8:  (3, 250, 16, 16, 6)
Class9:  (3, 250, 16, 16, 6)


In [24]:
def makeTrainingData(concateClass, label, class_num):
    counter = 0

    classImg = []
    classLabelList = []
    yClass = []

    for n in range(3):
        for d in range(concateClass.shape[1]):
            img1 = concateClass[n, d, :, :, :3]
            img2 = concateClass[n, d, :, :, 3:6]

            classImg.append(img1)
            classLabelList.append(f"{label}_{counter}")
            yClass.append(class_num)

            counter += 1

            classImg.append(img2)
            classLabelList.append(f"{label}_{counter}")
            yClass.append(class_num)

            counter += 1

    classImg = np.array(classImg)
    classLabelList = np.array(classLabelList)
    yClass = np.array(yClass)

    return classImg, classLabelList, yClass

In [31]:
class1Img, class1LabelList, yClass1 = makeTrainingData(concateClass= concateClass1, label= "C1", class_num= 0)
class2Img, class2LabelList, yClass2 = makeTrainingData(concateClass= concateClass2, label= "C2", class_num= 1)
class3Img, class3LabelList, yClass3 = makeTrainingData(concateClass= concateClass3, label= "C3", class_num= 2)
class4Img, class4LabelList, yClass4 = makeTrainingData(concateClass= concateClass4, label= "C4", class_num= 3)
class5Img, class5LabelList, yClass5 = makeTrainingData(concateClass= concateClass5, label= "C5", class_num= 4)
class6Img, class6LabelList, yClass6 = makeTrainingData(concateClass= concateClass6, label= "C6", class_num= 5)
class7Img, class7LabelList, yClass7 = makeTrainingData(concateClass= concateClass7, label= "C7", class_num= 6)
class8Img, class8LabelList, yClass8 = makeTrainingData(concateClass= concateClass8, label= "C8", class_num= 7)
class9Img, class9LabelList, yClass9 = makeTrainingData(concateClass= concateClass9, label= "C9", class_num= 8)


In [32]:
X_data = np.concatenate([
    class1Img,
    class2Img,
    class3Img,
    class4Img,
    class5Img,
    class6Img,
    class7Img,
    class8Img,
    class9Img
])

y_data = np.concatenate([
    yClass1,
    yClass2,
    yClass3,
    yClass4,
    yClass5,
    yClass6,
    yClass7,
    yClass8,
    yClass9
])

i_data = np.concatenate([
    class1LabelList,
    class2LabelList,
    class3LabelList,
    class4LabelList,
    class5LabelList,
    class6LabelList,
    class7LabelList,
    class8LabelList,
    class9LabelList
])

tmp = [[x, y, z] for x, y, z in zip(X_data, y_data, i_data)]

random.shuffle(tmp)

X_data = [n[0] for n in tmp]
y_data = [n[1] for n in tmp]
i_data = [n[2] for n in tmp]

npzDataPath = "./"
np.savez(os.path.join(npzDataPath, "Test0913"), X_data= X_data, y_data= y_data, i_data= i_data)